<a href="https://colab.research.google.com/github/gurujasko11/FAIS-deep-learning/blob/master/RecurrentNeuralNetworks/seq2seqWordLevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Embedding, Dense
from keras.utils import plot_model

import numpy as np

num_samples = 5000  # Number of samples to train on.

data_path = '/content/pol.txt'

Using TensorFlow backend.


In [0]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
lines_eng = []
lines_pol = []
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text, ignore = line.split('\t')[:3]
  lines_eng.append(input_text)
  lines_pol.append('START_ '+target_text+ ' _END')

In [0]:
all_eng_words=set()
for eng in lines_eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_pol_words=set()
for pol in lines_pol:
    for word in pol.split():
        if word not in all_pol_words:
            all_pol_words.add(word)

In [0]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_pol_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_pol_words)
max_lenght_eng = max([len(txt.split()) for txt in lines_eng])
max_lenght_pol = max([len(txt.split()) for txt in lines_pol])

input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [0]:
encoder_input_data = np.zeros(
    (len(lines_eng), max_lenght_eng),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(lines_pol), max_lenght_pol),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(lines_pol), max_lenght_pol, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(lines_eng, lines_pol)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [6]:
embedding_size = 50
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model
encoder_inputs = Input(shape=(None,))
# English words embedding
en_x =  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
# Encoder lstm
encoder = LSTM(150, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
# french word embeddings
dex =  Embedding(num_decoder_tokens, embedding_size)
final_dex = dex(decoder_inputs)
# decoder lstm
decoder_lstm = LSTM(150, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# While training, model takes eng and french words and outputs #translated french word
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# rmsprop is preferred for nlp tasks
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [7]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=50,
          validation_split=0.20)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 4000 samples, validate on 1000 samples
Epoch 1/50





4000/4000 [==============================] - 4s 1ms/step - loss: 2.0985 - acc: 0.0880 - val_loss: 2.2134 - val_acc: 0.0909
Epoch 2/50
4000/4000 [==============================] - 2s 575us/step - loss: 1.7402 - acc: 0.0964 - val_loss: 2.2235 - val_acc: 0.0992
Epoch 3/50
4000/4000 [==============================] - 2s 577us/step - loss: 1.6813 - acc: 0.0997 - val_loss: 2.2453 - val_acc: 0.0993
Epoch 4/50
4000/4000 [==============================] - 2s 581us/step - loss: 1.6314 - acc: 0.1005 - val_loss: 2.2361 - val_acc: 0.0994
Epoch 5/50
4000/4000 [==============================] - 2s 575us/step - loss: 1.5808 - acc: 0.1038 - val_loss: 2.2550 - val_acc: 0.1025
Epoch 6/50
4000/4000 [==============================] - 2s 575us/step - loss: 1.5285 - acc: 0.1060 - val_loss: 2.2214 - val_acc: 0.1040
Epoch 7/50
4000/4000 [==============

In [8]:
# define the encoder model 
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()
# Redefine the decoder model with decoder will be getting below inputs from encoder while in prediction
decoder_state_input_h = Input(shape=(150,))
decoder_state_input_c = Input(shape=(150,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
final_dex2= dex(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
# sampling model will take encoder states and decoder_input(seed initially) and output the predictions(french word index) We dont care about decoder_states2
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          144250    
_________________________________________________________________
lstm_1 (LSTM)                [(None, 150), (None, 150) 120600    
Total params: 264,850
Trainable params: 264,850
Non-trainable params: 0
_________________________________________________________________


In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
    # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
    # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True
    # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
    # Update states
        states_value = [h, c]
    return decoded_sentence

In [10]:
for seq_index in range(100, 200):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines_eng[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Welcome.
Decoded sentence:  Ptaki śpiewają. _END
-
Input sentence: Who ate?
Decoded sentence:  Kto to zrobić. _END
-
Input sentence: Who won?
Decoded sentence:  Kto to zrobić. _END
-
Input sentence: You run.
Decoded sentence:  Poproszę dalej. _END
-
Input sentence: You won.
Decoded sentence:  Jedz powoli. _END
-
Input sentence: Back off!
Decoded sentence:  Idź w wkrótce. _END
-
Input sentence: Back off.
Decoded sentence:  Proszę w dużo. _END
-
Input sentence: Be still.
Decoded sentence:  Bądź ostrożny. _END
-
Input sentence: Call Tom.
Decoded sentence:  Chodź tutaj. _END
-
Input sentence: Cheer up!
Decoded sentence:  Bądź ostrożny. _END
-
Input sentence: Cool off!
Decoded sentence:  Proszę w gruby. _END
-
Input sentence: Cuff him.
Decoded sentence:  Wyglądasz na dużo. _END
-
Input sentence: Drive on.
Decoded sentence:  Jedź dalej. _END
-
Input sentence: Find Tom.
Decoded sentence:  Pocałuj Toma. _END
-
Input sentence: Fix this.
Decoded sentence:  Weź to. _END
-
Input 